---
title: "第1讲：机器学习理论基础"
subtitle: "非时序任务的认知框架（i.i.d.）"
---


## 课程定位与本周目标

### 课程主线

本课程围绕**泛化—正则化—评估**三位一体展开：

- **泛化能力**：模型在未见数据上的表现，是机器学习的核心目标
- **正则化**：控制模型复杂度，防止过拟合的关键手段
- **评估规范**：科学衡量模型性能，确保研究结论可靠

### 本周聚焦

本周建立**i.i.d.（独立同分布）场景**下的"最小认知框架"，为后续金融应用打下理论基础。

:::{.callout-important}
**重要提示**：金融数据通常**不满足 i.i.d. 假设**（存在时序依赖、非平稳性等）。本周先理解基础框架，第2周将系统讲解金融时序评估规范。
:::

## 两类监督学习任务：回归 vs 分类

### 统一的符号系统

监督学习的基本设定：

- **特征向量** $\mathbf{x} \in \mathbb{R}^p$：描述观测对象的 $p$ 个属性
- **标签** $y$：我们希望预测的目标变量
  - 回归任务：$y \in \mathbb{R}$（连续值）
  - 分类任务：$y \in \{1, 2, \ldots, K\}$（离散类别）
- **训练集** $\{(\mathbf{x}_i, y_i)\}_{i=1}^n$：用于学习模型的已知数据
- **模型** $\hat{f}(\mathbf{x})$：从特征到预测的映射

### 回归任务（Regression）

#### 目标

预测一个**连续数值**，如股票收益率、信用评分、违约损失率等。

#### 常见损失函数

1. **均方误差（MSE）**：
$$
L(\hat{f}) = \frac{1}{n}\sum_{i=1}^n (y_i - \hat{f}(\mathbf{x}_i))^2
$$

2. **平均绝对误差（MAE）**：
$$
L(\hat{f}) = \frac{1}{n}\sum_{i=1}^n |y_i - \hat{f}(\mathbf{x}_i)|
$$

#### 典型模型输出

- 线性回归：$\hat{y} = \mathbf{x}^\top \boldsymbol{\beta}$
- 非线性回归：$\hat{y} = f(\mathbf{x})$（如神经网络、树模型等）

### 分类任务（Classification）

#### 目标

将观测对象归入**离散类别**，如预测涨跌方向、信用等级、欺诈与否等。

#### 常见损失函数

**对数损失（Log-loss / Cross-entropy）**：

对于二分类（$y \in \{0,1\}$）：
$$
L(\hat{f}) = -\frac{1}{n}\sum_{i=1}^n [y_i \log(\hat{p}_i) + (1-y_i)\log(1-\hat{p}_i)]
$$

其中 $\hat{p}_i = P(y_i=1|\mathbf{x}_i)$ 是模型预测的概率。

#### 典型模型输出

- **概率输出**：$\hat{p} = P(y=1|\mathbf{x}) \in [0,1]$
  - Logistic回归：$\hat{p} = \frac{1}{1+\exp(-\mathbf{x}^\top\boldsymbol{\beta})}$
- **类别预测**：$\hat{y} = \mathbb{I}(\hat{p} > \text{threshold})$

### 金融问题中的回归 vs 分类

:::{.callout-note icon=false}
📌 案例：预测股票未来收益

**同一问题，两种表述**：

**回归表述**：
- 标签：$y = r_{t+1}$（下期实际收益率）
- 预测：$\hat{r}_{t+1} = f(\mathbf{x}_t)$
- 评估指标：$R^2$, RMSE, IC（信息系数）

**分类表述**：
- 标签：$y = \mathbb{I}(r_{t+1} > 0)$（涨或跌）
- 预测：$\hat{p}_{t+1} = P(r_{t+1} > 0 | \mathbf{x}_t)$
- 评估指标：AUC, 精确率, 召回率

**选择依据**：
- 若关心收益大小的准确性 → 回归
- 若关心方向判断与排序 → 分类
- 若后续构建多空组合 → 两者皆可（收益预测或概率排序）
:::

## 偏差–方差分解与泛化误差

### 为什么模型会过拟合？

**核心问题**：训练集表现优异的模型，为何在新数据上表现很差？

答案在于**泛化误差的分解**。

### 偏差–方差分解（Bias-Variance Decomposition）

对于回归任务，测试点 $\mathbf{x}_0$ 处的期望预测误差可分解为：

$$
\mathbb{E}[(y_0 - \hat{f}(\mathbf{x}_0))^2] = \underbrace{[\mathbb{E}(\hat{f}(\mathbf{x}_0)) - f(\mathbf{x}_0)]^2}_{\text{偏差}^2} + \underbrace{\mathbb{E}[(\hat{f}(\mathbf{x}_0) - \mathbb{E}(\hat{f}(\mathbf{x}_0)))^2]}_{\text{方差}} + \underbrace{\sigma^2}_{\text{不可约误差}}
$$

#### 三个成分的含义

1. **偏差（Bias）**：
   - 模型的系统性误差
   - 反映模型的**拟合能力**
   - 过于简单的模型 → 高偏差（欠拟合）

2. **方差（Variance）**：
   - 模型对训练数据的敏感度
   - 反映模型的**稳定性**
   - 过于复杂的模型 → 高方差（过拟合）

3. **不可约误差（Irreducible Error）**：
   - 数据本身的噪声 $\sigma^2$
   - 任何模型都无法消除

### 过拟合的统计根源

过拟合本质上是**高方差**问题：

- 模型过于复杂，捕捉了训练数据中的噪声
- 在训练集上表现优异（低偏差，甚至零偏差）
- 在测试集上表现很差（高方差）

:::{.callout-warning}
⚠️ 金融数据中的过拟合陷阱

金融数据的**低信噪比**特点使过拟合问题尤为严重：

- 股票收益的可预测成分很小（$R^2$ 通常 < 5%）
- 复杂模型容易将噪声误认为信号
- 样本外表现往往远低于样本内
:::

### 学习曲线（Learning Curves）

**训练误差 vs 测试误差**随样本量的变化：


In [ ]:
#| echo: false
#| fig-cap: "学习曲线示意图"
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

n = np.linspace(50, 1000, 50)

# 欠拟合模型
train_error_underfit = 0.40 - 0.05 * np.log(n/50)
test_error_underfit = 0.45 - 0.05 * np.log(n/50)

# 良好拟合
train_error_good = 0.15 + 0.10 * np.log(n/50)
test_error_good = 0.30 - 0.05 * np.log(n/50)

# 过拟合模型
train_error_overfit = 0.02 + 0.03 * np.log(n/50)
test_error_overfit = 0.50 - 0.03 * np.log(n/50)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 欠拟合
axes[0].plot(n, train_error_underfit, label='训练误差', linewidth=2)
axes[0].plot(n, test_error_underfit, label='测试误差', linewidth=2)
axes[0].set_xlabel('训练样本量')
axes[0].set_ylabel('误差')
axes[0].set_title('欠拟合（高偏差）')
axes[0].legend()
axes[0].grid(alpha=0.3)

# 良好拟合
axes[1].plot(n, train_error_good, label='训练误差', linewidth=2)
axes[1].plot(n, test_error_good, label='测试误差', linewidth=2)
axes[1].set_xlabel('训练样本量')
axes[1].set_title('良好拟合')
axes[1].legend()
axes[1].grid(alpha=0.3)

# 过拟合
axes[2].plot(n, train_error_overfit, label='训练误差', linewidth=2)
axes[2].plot(n, test_error_overfit, label='测试误差', linewidth=2)
axes[2].set_xlabel('训练样本量')
axes[2].set_title('过拟合（高方差）')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

**关键观察**：

- **欠拟合**：训练误差和测试误差都很高，且接近
- **过拟合**：训练误差很低，但测试误差远高于训练误差
- **良好拟合**：两者适度分离，随样本增加而收敛

## 从 ERM 到 SRM：正则化与模型选择

### 经验风险最小化（ERM）

**标准监督学习**就是在训练集上最小化损失：

$$
\hat{f} = \arg\min_{f \in \mathcal{F}} \frac{1}{n}\sum_{i=1}^n L(y_i, f(\mathbf{x}_i))
$$

问题：当模型族 $\mathcal{F}$ 太丰富时，容易过拟合。

### 结构风险最小化（SRM）

**核心思想**：在损失函数中加入模型复杂度惩罚：

$$
\hat{f} = \arg\min_{f \in \mathcal{F}} \left[\frac{1}{n}\sum_{i=1}^n L(y_i, f(\mathbf{x}_i)) + \lambda \cdot \text{Complexity}(f)\right]
$$

其中 $\lambda > 0$ 是**正则化参数**，控制复杂度惩罚的强度。

### 线性模型中的正则化

对于线性回归 $\hat{y} = \mathbf{x}^\top \boldsymbol{\beta}$，常见正则化方法：

#### 岭回归（Ridge Regression）

$$
\hat{\boldsymbol{\beta}}^{\text{ridge}} = \arg\min_{\boldsymbol{\beta}} \left[\sum_{i=1}^n (y_i - \mathbf{x}_i^\top\boldsymbol{\beta})^2 + \lambda \sum_{j=1}^p \beta_j^2\right]
$$

**特点**：

- $L_2$ 惩罚：$\|\boldsymbol{\beta}\|_2^2$
- 系数**收缩**但不会变为零
- 适合特征间**多重共线性**严重的情况
- 闭式解：$\hat{\boldsymbol{\beta}}^{\text{ridge}} = (\mathbf{X}^\top\mathbf{X} + \lambda \mathbf{I})^{-1}\mathbf{X}^\top\mathbf{y}$

#### Lasso 回归

$$
\hat{\boldsymbol{\beta}}^{\text{lasso}} = \arg\min_{\boldsymbol{\beta}} \left[\sum_{i=1}^n (y_i - \mathbf{x}_i^\top\boldsymbol{\beta})^2 + \lambda \sum_{j=1}^p |\beta_j|\right]
$$

**特点**：

- $L_1$ 惩罚：$\|\boldsymbol{\beta}\|_1$
- 产生**稀疏解**：部分系数精确为零
- 具有**变量选择**功能
- 适合特征维度高、真正重要特征少的情况

#### 弹性网（Elastic Net）

$$
\hat{\boldsymbol{\beta}}^{\text{enet}} = \arg\min_{\boldsymbol{\beta}} \left[\sum_{i=1}^n (y_i - \mathbf{x}_i^\top\boldsymbol{\beta})^2 + \lambda_1 \sum_{j=1}^p |\beta_j| + \lambda_2 \sum_{j=1}^p \beta_j^2\right]
$$

**特点**：

- 结合 $L_1$ 和 $L_2$ 惩罚
- 在相关特征间**更稳定**
- 兼具变量选择与群组效应

:::{.callout-tip}
💡 金融应用中的选择建议

- **岭回归**：特征多重共线性严重（如多个估值指标）
- **Lasso**：特征数量远大于样本（如大量技术指标）
- **弹性网**：需要稳定的变量选择（如构建可解释的因子模型）
:::

### i.i.d. 假设下的数据划分

#### 训练-验证-测试三分法

```
原始数据
    ↓
┌──────────────────────────────────────────┐
│ 训练集 (60%)  │ 验证集 (20%) │ 测试集 (20%) │
└──────────────────────────────────────────┘
     ↓               ↓              ↓
  训练模型         选择超参数       最终评估
```

**职责分工**：

1. **训练集**：拟合模型参数（如 $\boldsymbol{\beta}$）
2. **验证集**：选择超参数（如 $\lambda$）与模型比较
3. **测试集**：仅用一次，报告最终泛化性能

:::{.callout-caution}
⚠️ 常见错误

**重复使用测试集**会导致"数据窥探"（data snooping），使测试性能失去公正性！
:::

#### K 折交叉验证（K-Fold Cross-Validation）

当数据量有限时，更充分利用数据：

```
原始数据切分为 K 份（如 K=5）：
┌────┬────┬────┬────┬────┐
│ 1  │ 2  │ 3  │ 4  │ 5  │
└────┴────┴────┴────┴────┘

Fold 1: [2,3,4,5] 训练, [1] 验证
Fold 2: [1,3,4,5] 训练, [2] 验证
Fold 3: [1,2,4,5] 训练, [3] 验证
Fold 4: [1,2,3,5] 训练, [4] 验证
Fold 5: [1,2,3,4] 训练, [5] 验证

最终性能 = 5 次验证误差的平均
```

:::{.callout-important}
🚨 金融数据的特殊性

**K 折交叉验证仅适用于 i.i.d. 数据！**

金融时序数据必须用**滚动窗口**或**扩展窗口**评估（详见第2周）。
:::

## 关键评估指标体系

模型好坏必须用**与研究目标一致**的指标衡量。

## 回归任务指标

#### 均方误差（MSE）与均方根误差（RMSE）

$$
\text{MSE} = \frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2, \quad \text{RMSE} = \sqrt{\text{MSE}}
$$

- RMSE 与 $y$ 同量纲，更易解释

#### 平均绝对误差（MAE）

$$
\text{MAE} = \frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|
$$

- 对异常值更鲁棒

#### 决定系数（$R^2$）

$$
R^2 = 1 - \frac{\sum_{i=1}^n (y_i - \hat{y}_i)^2}{\sum_{i=1}^n (y_i - \bar{y})^2}
$$

- 衡量模型解释的方差比例
- 样本内 $R^2$ 总是 $\leq 1$

#### 样本外 $R^2$（Out-of-Sample $R^2$）

$$
R^2_{\text{OOS}} = 1 - \frac{\sum_{i \in \text{test}} (y_i - \hat{y}_i)^2}{\sum_{i \in \text{test}} (y_i - \bar{y}_{\text{train}})^2}
$$

- **可能为负**（模型比历史均值还差）
- 金融资产定价中的关键指标

### 分类任务指标

分类问题的评估指标可分为两大类：**基于混淆矩阵的指标**（依赖于特定阈值）与 **AUC-ROC 指标**（与阈值无关）。

#### 混淆矩阵（Confusion Matrix）

对于二分类问题，给定分类阈值（如 $\hat{p} > 0.5$），可构建混淆矩阵：

```
                预测结果
              正类    负类
实际  正类    TP     FN
结果  负类    FP     TN
```

其中：

- **TP（True Positive）**：真正例，正确预测为正类
- **TN（True Negative）**：真负例，正确预测为负类
- **FP（False Positive）**：假正例，错误预测为正类（第一类错误）
- **FN（False Negative）**：假负例，错误预测为负类（第二类错误）

#### 基于混淆矩阵的指标（依赖阈值）

##### 1. 准确率（Accuracy）

$$
\text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}
$$

- **含义**：分类正确的样本占总样本的比例
- **局限**：类别不平衡时容易误导（如违约率 1% 时，全预测为"不违约"准确率仍达 99%）

##### 2. 精确率（Precision）

$$
\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}
$$

- **含义**：预测为正类的样本中，真正为正类的比例
- **应用**：关心误报成本时（如垃圾邮件过滤）

##### 3. 召回率（Recall / Sensitivity / TPR）

$$
\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}
$$

- **含义**：真实正类中，被正确识别的比例
- **应用**：关心漏报成本时（如疾病诊断、欺诈检测）

##### 4. F1 分数（F1-Score）

$$
F_1 = 2 \cdot \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} = \frac{2 \cdot \text{TP}}{2 \cdot \text{TP} + \text{FP} + \text{FN}}
$$

- **含义**：精确率与召回率的调和平均
- **特点**：平衡两者，单一指标综合评价
- **适用**：类别不平衡且需平衡误报/漏报时

:::{.callout-warning}
⚠️ 阈值依赖性问题

以上指标都**依赖于分类阈值**的选择。不同阈值下，这些指标会有不同的值：

- 提高阈值 → Precision ↑, Recall ↓
- 降低阈值 → Precision ↓, Recall ↑

因此，单一阈值下的指标**无法全面反映模型的分类能力**。
:::

#### AUC-ROC：与阈值无关的评估指标

##### ROC 曲线（Receiver Operating Characteristic Curve）

**ROC 曲线**绘制**所有可能阈值**下的分类表现，横轴为假正例率（FPR），纵轴为真正例率（TPR）：

$$
\text{TPR} = \frac{\text{TP}}{\text{TP}+\text{FN}} \quad (\text{即 Recall})
$$

$$
\text{FPR} = \frac{\text{FP}}{\text{FP}+\text{TN}} \quad (\text{负类中的误报率})
$$

**绘制过程**：

1. 将所有样本按预测概率 $\hat{p}$ 降序排列
2. 依次将每个样本的 $\hat{p}$ 作为阈值
3. 计算每个阈值下的 (FPR, TPR) 坐标点
4. 连接所有点形成曲线

##### AUC（Area Under Curve）

**AUC** 是 ROC 曲线下的面积：

- **取值范围**：[0, 1]，越大越好
- **AUC = 0.5**：随机猜测（对角线）
- **AUC = 1.0**：完美分类器
- **AUC < 0.5**：比随机猜测还差（反向预测）

**核心优势：与阈值无关**

AUC 衡量模型的**排序能力**，而非特定阈值下的分类表现：

- **概率解释**：随机选取一对样本（正例，负例），模型对正例打分更高的概率
- **不受阈值影响**：即使不知道最佳阈值，AUC 仍能评估模型质量
- **不受类别比例影响**：相比准确率，AUC 对类别不平衡更鲁棒

**应用场景**：

- 金融风控：预测违约概率，关心**风险排序**而非具体阈值
- 推荐系统：关心**排序质量**，推荐高分项目
- 医学诊断：关心**筛查能力**，后续人工复查确定阈值

:::{.callout-tip}
💡 何时使用 AUC vs F1？

- **AUC-ROC**：评估模型的整体排序/区分能力，适合比较不同模型
- **F1 分数**：业务已确定决策阈值，需要在该阈值下评估性能
- **实践建议**：先用 AUC 选择模型，再根据业务需求调整阈值优化 F1
:::

#### 补充指标：AUC-PR（Precision-Recall 曲线）

**PR 曲线**：横轴为召回率（Recall），纵轴为精确率（Precision）

**适用场景**：

- 类别**极度不平衡**时（如违约率 0.5%，欺诈率 0.1%）
- 此时 ROC 曲线可能过于"乐观"（TN 很大，FPR 很小）
- PR 曲线聚焦于正类预测质量

:::{.callout-tip}
### 💡 指标选择的黄金法则

**先明确研究目标，再选指标**：

- 关心排序 → AUC-ROC
- 关心不平衡分类 → AUC-PR, F1
- 关心数值预测 → MSE, MAE, $R^2$
:::

## 类别不平衡的常见误区

**案例**：预测信用违约，违约率 1%。

❌ **错误做法**：

```python
# 模型预测所有人都不违约
accuracy = 0.99  # 准确率 99%！
```

虽然准确率很高，但模型毫无用处！

✅ **正确做法**：

- 使用 AUC-PR、F1、召回率等指标
- 考虑代价敏感学习
- 调整决策阈值以平衡误报与漏报

## 机器学习的最小证据链

### 什么是"最小证据链"？

**定义**：从原始数据到研究结论，**可审计、可复现、可信任**的完整证据路径。

:::{.callout-note icon=false}
📋 最小证据链检查清单

1. ✅ **数据与标签定义**
   - 数据来源、版本、时间范围
   - 标签如何定义（含信息可得性说明）
   - 缺失值、异常值如何处理

2. ✅ **预处理规则**
   - 标准化、归一化方式
   - 特征工程步骤
   - 缺失值填充策略

3. ✅ **划分与评估策略**
   - 训练-验证-测试如何划分
   - i.i.d. 还是时序评估
   - 超参数选择方法

4. ✅ **基准模型**
   - 简单基准（历史均值、线性模型等）
   - 与既有文献的对比

5. ✅ **样本外结果**
   - 主要指标与置信区间
   - 切片评估（按时间、行业等）
   - 稳健性检验（换特征、换窗口等）

6. ✅ **可复现性材料**
   - 代码与随机种子
   - 变量字典与流程图
   - 环境配置说明
:::

### 为什么强调"最小"？

- **可行性**：要求过高会降低研究效率
- **聚焦核心**：抓住信息泄露、不可复现等关键风险点
- **可审计**：评审者/读者能快速验证研究可信度

:::{.callout-warning}
⚠️ 常见缺失环节

1. **信息可得性说明不清**：用了未来数据或不可交易的信息
2. **预处理不透明**：标准化用了全局统计量（含测试集信息）
3. **超参数选择不规范**：在测试集上调参
4. **只报最好结果**：隐藏失败尝试，过度拟合
5. **无代码或种子**：结果无法复现
:::

## 本周小结

### 核心要点

1. **监督学习框架**：回归 vs 分类，损失函数决定模型行为
2. **过拟合根源**：偏差–方差权衡，复杂模型 + 噪声数据 → 高方差
3. **正则化**：岭/Lasso/弹性网，控制复杂度，提升泛化能力
4. **评估规范**：训练-验证-测试分工，指标与目标一致
5. **最小证据链**：可审计、可复现的研究规范

### 本周思考题

#### 问题 1

将"预测未来收益率"分别表述为回归任务与分类任务时，标签应如何定义？两种表述分别应优先使用哪些评价指标？

:::{.callout-note collapse="true"}
💡 参考答案

**回归表述**：

- 标签：$y = r_{t+h}$（未来 $h$ 期的实际收益率）
- 优先指标：MSE、MAE、$R^2$、样本外 $R^2$、IC（信息系数）

**分类表述**：

- 标签：$y = \mathbb{I}(r_{t+h} > \text{threshold})$（如涨跌、超额收益正负）
- 优先指标：AUC-ROC、Log-loss、Rank-IC（排序相关性）

**选择依据**：若后续构建组合，**分类表述的排序能力**往往更重要。
:::

#### 问题 2

用偏差–方差分解解释：为什么更复杂的模型可能在训练集更好，但在样本外更差？

:::{.callout-note collapse="true"}
💡 参考答案

- **训练集**：复杂模型可降低偏差（更灵活拟合），训练误差↓
- **测试集**：复杂模型方差高（对训练数据过敏感），捕捉噪声
- **结果**：测试误差 = 偏差² + 方差 + 不可约误差，方差项主导↑

**关键**：训练误差只反映偏差，不反映方差！
:::

#### 问题 3

请列出"最小证据链"中你认为最容易被忽视的两项，并说明忽视它们可能带来的误判。

:::{.callout-note collapse="true"}
💡 参考答案示例

1. **信息可得性说明**
   - 忽视后果：使用未来数据（look-ahead bias），样本内指标虚高
   - 实例：用当期年报数据预测当期收益（年报披露有滞后）

2. **随机种子与环境配置**
   - 忽视后果：结果无法复现，无法验证真实性
   - 实例：不同机器/版本得到不同结果，研究可信度存疑
:::

---

### 下周预告

第2周将进入**金融场景的特殊性**：

- 时序数据如何评估（滚动窗口、嵌套CV）
- 回测卫生清单（存活偏误、信息泄露等）
- 从预测到交易：组合构建与成本后评估

请提前阅读：

- Gu, Kelly & Xiu (2020) *Empirical asset pricing via machine learning*
- Kelly & Xiu (2023), Chapters 3-5

**预习提示**：在文中找出"预测表现好但不可交易"的证据或讨论点，带到课上！